<h4>Features 14/06/2024</h4>

<p>Yet another revision of features with latest ideas of mine</p>

In [1]:
from pathlib import Path
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import *
from tqdm import tqdm


import polars as pl
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


ROOT_DIR = Path(os.getcwd()).parent.parent

In [2]:
@dataclass
class PumpEvent:
    pump_id: int
    ticker: str
    time: str
    exchange: str

    def __post_init__(self):
        self.time: pd.Timestamp = pd.Timestamp(self.time)

    def __str__(self):
        return f"Pump event: {self.ticker} - {str(self.time)} on {self.exchange}"

In [3]:
# data is organized by days
pump = PumpEvent(
    pump_id=1, ticker="AGIXBTC", time="2022-08-14 16:00:05", exchange="binance"
)

def load_data(pump: PumpEvent, lookback_delta: timedelta) -> pd.DataFrame:

    end: pd.Timestamp = pump.time.round("1h") - timedelta(hours=1)
    start: pd.Timestamp = end - lookback_delta
    
    date_range: List[pd.Timestamp] = pd.date_range(
        start=start,
        end=end, freq="D",
        inclusive="both"
    ).tolist()

    df: pd.DataFrame = pd.DataFrame()

    for date in tqdm(date_range):
        file_name: str = f"{pump.ticker}-trades-{date.date()}.parquet"
        df_date: pd.DataFrame = pd.read_parquet(
            os.path.join(ROOT_DIR, f"data/trades_parquet/{pump.exchange}/{pump.ticker}", file_name)
        )
        
        df = pd.concat([df, df_date])

    df["time"] = pd.to_datetime(df["time"], unit="ms")
    df = df[(df["time"] >= start) & (df["time"] <= end)].reset_index(drop=True)

    return df

<h4>Load data</h4>

<p>We will need at least a month worth of data as I will want to compare metrics computed in the last 24h-7d before the pump with past values</p>

In [4]:
df = load_data(pump=pump, lookback_delta=timedelta(days=30))
df["quote"] = df["qty"] * df["price"]

100%|██████████| 31/31 [00:00<00:00, 88.65it/s]


In [5]:
df["qty_sign"] = (1 - 2 * df["isBuyerMaker"]) * df["qty"]
df["quote_sign"] = (1 - 2 * df["isBuyerMaker"]) * df["quote"]

# Aggregate by time into rush orders
df_trades: pd.DataFrame = df.groupby("time").agg(
    price_first=("price", "first"),
    price_last=("price", "last"),
    price_max=("price", "max"),
    price_min=("price", "min"),
    qty_sign=("qty_sign", "sum"),
    qty_abs=("qty", "sum"),
    quote_sign=("quote_sign", "sum"),
    quote_abs=("quote", "sum"),
)

df_trades["is_long"] = df_trades["qty_sign"] >= 0 # is buyer
df_trades["quote_long"] = df_trades["quote_abs"] * df_trades["is_long"] # quote volume for longs
df_trades["quote_short"] = df_trades["quote_abs"] * ~df_trades["is_long"] # quote volume for shorts
 
df_trades = df_trades.reset_index()
df_trades.head(2)

,time,price_first,price_last,price_max,price_min,qty_sign,qty_abs,quote_sign,quote_abs,is_long,quote_long,quote_short
0,2022-07-15 15:00:02.422,0.000002,0.000002,0.000002,0.000002,-2437.0,2437.0,-0.004240,0.004240,False,0.000000,0.00424
1,2022-07-15 15:00:02.434,0.000002,0.000002,0.000002,0.000002,275.0,275.0,0.000479,0.000479,True,0.000479,0.00000


<h4>Slippages</h4>

$$
\text{Slippage Loss} = \underbrace{\sum_{i=1}^{N}{\text{qty\_sign}_i} \cdot P_i}_{\text{Quote actually spent}} \: - \underbrace{\sum_{i=1}^N{\text{qty\_sign}_i} \cdot P_0}_{\substack{\text{Quote could have been spent} \\ \text{if filled at best price}}}
$$

This formula calculates the slippage loss as the difference between quote we actually spent and quote amount we would have paid if we were able to execute at the best bid or ask price (depending on the side of the trade)

In [11]:
# calculate slippages
df_trades["quote_slippage_abs"] = np.abs(
    df_trades["quote_abs"] - df_trades["qty_abs"] * df_trades["price_first"]
)
df_trades["quote_slippage_sign"] = df_trades["quote_slippage_abs"] * np.sign(df_trades["qty_sign"])
df_trades["quote_slippage_long"] = df_trades["quote_slippage_abs"] * df_trades["is_long"]

<h4>Hourly candlestick features</h4>

In [13]:
df_hourly_candles: pd.DataFrame = (
    df_trades
    .resample(on="time", rule="1h", closed="left")
    .agg(
        open=("price_first", "first"),
        close=("price_last", "last"),
        low=("price_min", "min"),
        high=("price_max", "max"),
        volume_qty_abs=("qty_abs", "sum"), # absolute volume in base asset
        volume_quote_abs=("quote_abs", "sum"), # absolute volume in quote asset
        volume_quote_abs_long=("quote_long", "sum"),
        num_trades=("is_long", "count"), 
        num_trades_long=("is_long", "sum"), 
        quote_slippage_abs=("quote_slippage_abs", "sum"), # slippage loss incurred by both buy and sell sides
        quote_slippage_abs_long=("quote_slippage_long", "sum") # quote slippage incurred by longs
    )
).reset_index()

df_hourly_candles["log_return"] = np.log(
    df_hourly_candles["close"] / df_hourly_candles["close"].shift(1)
)

In [14]:
df_hourly_candles.tail(1)

,time,open,close,low,high,volume_qty_abs,volume_quote_abs,volume_quote_abs_long,num_trades,num_trades_long,quote_slippage_abs,quote_slippage_abs_long,log_return
719,2022-08-14 14:00:00,0.000003,0.000003,0.000003,0.000003,2358663.0,6.093561,2.671495,223,133,0.010421,0.008779,0.031131


<h4>Binned features</h4>

<p>Following methodology of Xu and Livshits 2019, we calculate overall log return and hourly log returns volatility over different windows</p>

We will scale absolute features like volumes and quote quantities by long run std

In [15]:
time_ub: pd.Timestamp = pump.time.round("1h") - timedelta(hours=1)

# long run mean and std of volumes in base and quote
df_hourly_lr: pd.DataFrame = df_hourly_candles[
    df_hourly_candles["volume_quote_abs"] <= df_hourly_candles["volume_quote_abs"].quantile(.99)
].copy()

<h4>Log returns and volume features</h4>

In [16]:
hour_offsets: List[int] = [1, 6, 24, 48, 72, 7*24, 14*24]

hourly_features: Dict[str, float] = {}

for offset in hour_offsets:
    df_window: pd.DataFrame = df_hourly_candles[df_hourly_candles["time"] >= time_ub - timedelta(hours=offset)].copy()
    
    hourly_features[f"overall_return_{offset}h"] = (df_window["log_return"] + 1).prod() # overall return if held for the whole window up to the last hour

    # Scaled volumes in base and quote assets
    hourly_features[f"volume_quote_abs_zscore_{offset}h_30d"] = (
        (df_window["volume_quote_abs"].mean() - df_hourly_lr["volume_quote_abs"].mean()) / df_hourly_lr["volume_quote_abs"].std()
    )

    # hourly_features[f"num_trades_long_share_{offset}h"] = df_window["num_trades_long"].sum() / df_window["num_trades"].sum()
    hourly_features[f"volume_quote_long_share_{offset}h"] = df_window["volume_quote_abs_long"].sum() / df_window["volume_quote_abs"].sum()

    if offset == 1:
        continue
    # Hourly log returns volatility scaled by long run volatility
    hourly_features[f"log_return_std_{offset}h_30d"] = np.log(df_window["log_return"].std() / df_hourly_lr["log_return"].std())
    # hourly log returns mean scaled by long run std -> z-score
    hourly_features[f"log_return_zscore_{offset}h_30d"] = df_window["log_return"].mean() / df_hourly_lr["log_return"].std() 

In [ ]:
len(hourly_features)

<h4>Slippage features</h4>

In [ ]:
slippage_features: Dict[str, float] = {}

df_hourly_candles_120h: pd.DataFrame = df_hourly_candles[
    df_hourly_candles["time"] >= time_ub - timedelta(hours=120)
].copy()

hour_offsets: List[int] = [1, 6, 24, 48, 72]

for offset in hour_offsets:
    df_window: pd.DataFrame = df_hourly_candles[df_hourly_candles["time"] >= time_ub - timedelta(hours=offset)].copy()
    # Share of overall slippages of this time window in 120hours
    slippage_features[f"quote_slippage_abs_share_{offset}h_120h"] = (
        df_window["quote_slippage_abs"].sum() / df_hourly_candles_120h["quote_slippage_abs"].sum()
    )

len(slippage_features)

<h4>Imbalance features</h4>

In [ ]:
imbalance_features: Dict[str, float] = {}

for offset in hour_offsets:
    df_window: pd.DataFrame = df_trades[df_trades["time"] >= time_ub - timedelta(hours=offset)].copy()
    # Volume imbalance ratio to see if there is more buying pressure
    imbalance_features[f"quote_imbalance_ratio_{offset}h"] = df_window["quote_sign"].sum() / df_window["quote_abs"].sum()
    # Imbalance ratio in slippages to see if there is skew towards long slippages
    imbalance_features[f"quote_slippage_imbalance_ratio_{offset}h"] = df_window["quote_slippage_sign"].sum() / df_window["quote_slippage_abs"].sum()

len(imbalance_features)

<h4>EVT features</h4>

In [ ]:
df_trades[
    df_trades["time"] >= time_ub - timedelta(days=3)
].plot.scatter(x="time", y="quote_sign", figsize=(20, 4))

plt.show()

In [ ]:
from scipy.stats import powerlaw


evt_features: Dict[str, float] = {}


for offset in hour_offsets:
    df_window: pd.DataFrame = df_trades[df_trades["time"] >= time_ub - timedelta(hours=offset)].copy()

    evt_features[f"quote_abs_powerlaw_alpha_{offset}h"] = powerlaw.fit(
        df_window[df_window["quote_abs"] >= df_window["quote_abs"].quantile(.99)]["quote_abs"]
    )[0]

len(evt_features)

<h4>Powerlaw manual estimation</h4>

We want to use EVT features that could allow us to capture heavy tails

In [ ]:
cutoff = df_trades["quote_abs"].quantile(.99)
    
# Filter tails
df_filtered = df_trades[df_trades["quote_abs"] >= cutoff].copy()

# Create log space bins
logbins = np.logspace(
    np.log10(cutoff), np.log10(df_trades["quote_abs"].max()), num=100
)

# Calculate PDF
hist, bin_edges = np.histogram(df_filtered["quote_abs"], bins=logbins, density=True)

df_pdf = pd.DataFrame({
    "quote_abs": bin_edges[:-1],
    "p": hist
})

df_pdf = df_pdf[
    (df_pdf["quote_abs"] >= cutoff) & (df_pdf["p"] != 0)
].copy()

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant


X = np.log10(df_pdf["quote_abs"])
X = add_constant(X)

Y = np.log10(df_pdf["p"])

model = OLS(
    endog=Y, exog=X
).fit()

print(model.summary())

In [ ]:
intercept, slope = model.params

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 6))
ax1, ax2 = axs

# First plot
X = np.log10(df_pdf["quote_abs"])
Y = np.log10(df_pdf["p"])

ax1.scatter(X, Y, alpha=.3, label="Observed PDF")

Y = intercept + slope * X

ax1.set_title("Log scaled. Powerlaw fitted to observed data")
ax1.set_xlabel("log(quote_abs)")
ax1.set_ylabel("log(Estimated PDF)")

ax1.plot(X, Y, color="red", label="Fitted PDF")
ax1.legend()

# Second plot
X = df_pdf["quote_abs"]
Y = df_pdf["p"]

a = 10**intercept
b = slope

ax2.scatter(X, Y, alpha=.3, label="Observed PDF")
ax2.plot(X, a*X**b, color="red", label="Fitted PDF")

ax2.set_title("Powerlaw fitted to observed data")
ax2.set_xlabel("quote_abs")
ax2.set_ylabel("Estimated PDF")

ax2.legend()

fig.tight_layout()
fig.savefig("imgs/powerlaw.png", transparent=True)

$$\log{f} = \underbrace{\log{\alpha}}_{\text{intercept}} + \underbrace{(\alpha - 1)}_{\text{slope}}\log{x}$$

In [ ]:
plt.figure(figsize=(10, 6))

X = df_pdf["quote_abs"]
Y = df_pdf["p"]

a = 10**intercept
b = slope

plt.scatter(X, Y, alpha=.3, label="Observed PDF")

plt.plot(X, a*X**b, color="red", label="Fitted PDF")

plt.show()